In [ ]:
#  ASR Notes 
# ==========================
#     1/ Store S&P symbols for ALL years in one File , so that easy to Load into dataFramme and slice it
#     2/ gains also store in One FILE for ALL Years , so that we can plot and compare multiple yeras
    
# StackOverFlow good links:
# =============================
# 1/ DF Copy:  http://stackoverflow.com/questions/34682828/pandas-extracting-specific-selected-columns-from-a-dataframe-to-new-dataframe   
  
# 2/ Adding a Row : http://stackoverflow.com/questions/27873190/creating-new-pandas-dataframe-from-existing-dataframe-and-index
        

In [38]:
#import desired packages
import numpy as np
import pandas as pd
from pprint import pprint
import seaborn as sns
import matplotlib.pyplot as plt
import time
%matplotlib inline 

In [39]:
import urllib.request

def get_year_open_close_price(symbol, year):
    u = "http://chart.finance.yahoo.com/table.csv?s={}&a={}&b={}&c={}&d={}&e={}&f={}&g=d&ignore=.csv"
    month, day = 1, 1  # JAN 1 , first day
    u1 = u.format(symbol, month-1, day, year, month-1, day+5, year)
    
    month, day = 12, 31  # Dec 31 , last day
    u2 = u.format(symbol, month-1, day-5, year, month-1, day, year)
    
    try:
        urllib.request.urlretrieve (u1, "mydat.csv")
        df = pd.read_csv('mydat.csv', header=0)
        price1 = float(df['Open'].tolist()[-1])  # why -1 ? to get first record value by Date
    except: 
        print ('no Data for {} , for the year {}'.format(symbol, year))
        return []  # retrun empty in case of exception

    urllib.request.urlretrieve (u2, "mydat.csv")
    df = pd.read_csv('mydat.csv', header=0)
    price2 = float(df['Close'].tolist()[0])  # why 0 ? to get last record value ny Date
    
    return [ year, symbol, round(price1, 2), round(price2, 2),  round(((price2 - price1) * 100)/price1, 2)]
   


In [58]:
for year in range(2008, 2009):
    rec = get_year_open_close_price('IBM', year)
    print (rec)

[2008, 'IBM', 108.99, 84.16, -22.78]


In [68]:
#import SnP Symbol list 
snp_tickers_allyears = pd.read_csv('data-SnP-10years-Tickers-byyear-dfready.csv', header=0)

gains_lst = []
for year in (2008,2008):
     
    llst = snp_tickers_allyears.query('Year == @year').values.tolist()
    #llst = [ ['dd', 'dddd'], ['AAPL', 2008]]
    for stock in llst[1:] :  # skip 0 th element being it header
        # here index are : 0 - TIcker , 1 - year
        #print('trying to fetch data for ' + stock[0])
        try:
            rec = get_year_open_close_price(stock[0] , stock[1])  
            if len(rec) > 0:
                gains_lst.append(rec)
                #print( rec[1] + ' is good')
            else:
                print(stock[0])
        except:
            a = 2
            #print('Exception for { }, year {}'.format(stock[0], stock[1])

df = pd.DataFrame(gains_lst)
cols = ['Year', 'Symbol', 'Open', 'Close', 'Gain']
df.columns = cols
df2 = df.sort_values(by='Gain', ascending=False)
df2.to_csv('data-SnP-TOP-gains-', index=False)


no Data for ABI , for the year 2008
ABI
no Data for ACE , for the year 2008
ACE
no Data for ALTR , for the year 2008
ALTR
no Data for AW , for the year 2008
AW
no Data for AYE , for the year 2008
AYE
no Data for BF.B , for the year 2008
BF.B
no Data for BJS , for the year 2008
BJS
no Data for BNI , for the year 2008
BNI
no Data for BRCM , for the year 2008
BRCM
no Data for BTU , for the year 2008
BTU
no Data for BUD , for the year 2008
BUD
no Data for CC , for the year 2008
CC
no Data for CIT , for the year 2008
CIT
no Data for COV , for the year 2008
COV
no Data for CTX , for the year 2008
CTX
no Data for CZR , for the year 2008
CZR
no Data for DTV , for the year 2008
DTV
no Data for DYN , for the year 2008
DYN
no Data for EDS , for the year 2008
EDS
no Data for EK , for the year 2008
EK
no Data for EP , for the year 2008
EP
no Data for EQ , for the year 2008
EQ
no Data for FDO , for the year 2008
FDO
no Data for FNM , for the year 2008
FNM
no Data for GAS , for the year 2008
GAS
no D

"\ndf = pd.DataFrame(gains_lst)\ncols = ['Year', 'Ticker', 'Open', 'Close', 'Gain']\ndf.columns = cols\ngains_df = df.sort(['Gain'], ascending=False)\ngains_df.to_csv( 'data-SnP-TOP-gains-', index=False)      \n"

In [66]:
def rescue_code(function):
    import inspect
    get_ipython().set_next_input("".join(inspect.getsourcelines(function)[0]))

In [26]:
df2 = df.sort_values(by='Gain', ascending=False) 
df3 = df2.query('Gain > 80 or Gain < -40')
df3

NameError: name 'df' is not defined

In [15]:
def get_SnP_symbols_for_year( year):
    
    try:
        df = pd.read_csv('sp-add-remove-final.csv', header=0)
        df2 = df.query(' Year == @year')
        return df2
    except: 
        return []  # retrun empty in case of exception
    
df3 = get_SnP_symbols_for_year (2016)
df3

,Date,Added,Removed,Year
1,12/01/2016,MAA,OI,2016
2,12/01/2016,EVHC,LM,2016
3,09/30/2016,COTY,DO,2016
4,09/22/2016,COO,HOT,2016
5,09/08/2016,CHTR,EMC,2016
6,09/06/2016,MTD,JCI,2016
7,07/05/2016,FTV,CPGX,2016
8,07/01/2016,LNT,GAS,2016
9,07/01/2016,ALB,TE,2016
10,06/22/2016,FBHS,CVC,2016


In [32]:
import traceback

def create_SnP_symbols_by_year():
    
    try:
        df = pd.read_csv('SnP-10-years.csv', header=0)
        df.to_csv('SnP-10-years-dfready.csv', index=False)  # use this CREATEd DF Ready file for later use
        return True
    except: 
        traceback.print_exc()
        return False  # retrun empty in case of exception
    
ret = create_SnP_symbols_by_year()
print(ret)

True
